In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

#Локальный путь. Используем, если скачали модель локально ранее
#base_model = "models/download/pythia-12b-deduped"

#Скачиваем базовую модель с Hugging Face 
base_model = "EleutherAI/pythia-12b-deduped"

#Локальный путь к peft модели. 
#Поменяйте на свой путь, указанный при сохранении модели по итогам работы тетрадки 01_peft_finetune
peft_model = "models/trained/peft/"

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True, 
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(peft_model)

In [ ]:
#Загружаем PEFT модель
from peft import PeftModel

model = PeftModel.from_pretrained(model, peft_model)

In [ ]:
from transformers import GenerationConfig

def generate_prompt(instruction):
    return f"""Вопрос: {instruction} Ответ: """

#Можно тюнить параметры. Влияет на качество ответов
def evaluate(
        instruction,
        temperature=0.8,
        top_p=0.75,
        top_k=40,
        num_beams=3,
        **kwargs,
):
    prompt = generate_prompt(instruction)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        no_repeat_ngram_size=7,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=512,
            
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output

prompt = "Что такое VK Cloud?"
output = evaluate(prompt)
print(output.strip())

In [ ]:
#Меняем текст вопроса в prompt на свой
prompt = "Какие сервисы предоставляет VK Cloud?"
output = evaluate(prompt)
print(output.strip())

In [ ]:
#Далее демонстрация работы модели в различных задачах
#Ответы на вопросы, краткое изложение текстов, ответы с учетом контекста
prompt = "Что такое Kubernetes?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Расскажи подробно про Kubernetes?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Как получить список подов в Kubernetes?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Что такое etcd в Kubernetes?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Расскажи подробно про эту команду kubectl get pods"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Как создать файл в Linux?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Как удалить файл в Linux?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Что такое ssh?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Напиши функцию на python для определения четное число или нет?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Напиши функцию на python для подсчета числа символов в слове?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Напиши программу Калькулятор на python, которая принимает два числа и операцию (+, -, *, /) от пользователя, а затем выполняет операцию и выводит результат"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = "Напиши функцию на python, которая выводит числа Фибоначчи?"
output = evaluate(prompt)
print(output)

In [ ]:
prompt = """Изложи кратко этот текст, используй данные только из этого текста: Алекса́ндр Серге́евич Пу́шкин — русский поэт, драматург и прозаик, заложивший основы русского реалистического направления, литературный критик и теоретик литературы, историк, публицист, журналист. Один из самых авторитетных литературных деятелей первой трети XIX века. Википедия
Дата и место рождения: 6 июня 1799 г., Москва, Россия"""
output = evaluate(prompt)
print(output)

In [ ]:
prompt = """Когда родился Пушкин? Алекса́ндр Серге́евич Пу́шкин — русский поэт, драматург и прозаик, заложивший основы русского реалистического направления, литературный критик и теоретик литературы, историк, публицист, журналист. Один из самых авторитетных литературных деятелей первой трети XIX века. Википедия
Дата и место рождения: 6 июня 1799 г., Москва, Россия"""
output = evaluate(prompt)
print(output)

In [ ]:
prompt = """Извлеки самые важные слова из этого текста: VK Cloud — вендор платформы облачных сервисов и технологический партнёр, который помогает компаниям строить и 
модернизировать ИТ-инфраструктуру для большей надёжности, ускорения разработки и оптимизации расходов. 
Безопасная и масштабируемая платформа облачных сервисов от VK позволяет строить надежные ИТ-системы для 
обслуживания бизнеса, разработки и предоставления сервисов.
Входящие в VK Cloud решения — виртуальные сервера, объектное хранилище, управляемые базы данных, 
кластеры Kubernetes и многое другое— позволяют запускать приложения и сервисы, работать с большими объемами данных и 
решать другие задачи крупного, среднего и малого бизнеса."""

output = evaluate(prompt)
print(output)

In [ ]:
prompt = """Изложи кратко этот текст: VK Cloud — вендор платформы облачных сервисов и технологический партнёр, который помогает компаниям строить и 
модернизировать ИТ-инфраструктуру для большей надёжности, ускорения разработки и оптимизации расходов. 
Безопасная и масштабируемая платформа облачных сервисов от VK позволяет строить надежные ИТ-системы для 
обслуживания бизнеса, разработки и предоставления сервисов.
Входящие в VK Cloud решения — виртуальные сервера, объектное хранилище, управляемые базы данных, 
кластеры Kubernetes и многое другое— позволяют запускать приложения и сервисы, работать с большими объемами данных и 
решать другие задачи крупного, среднего и малого бизнеса."""

output = evaluate(prompt)
print(output)

In [ ]:
prompt = """Где родился Пушкин? Алекса́ндр Серге́евич Пу́шкин — русский поэт, драматург и прозаик, заложивший основы русского реалистического направления, литературный критик и теоретик литературы, историк, публицист, журналист. Один из самых авторитетных литературных деятелей первой трети XIX века. Википедия
Дата и место рождения: 6 июня 1799 г., Москва, Россия"""
output = evaluate(prompt)
print(output)

In [ ]:
prompt = """Вопрос: Изложи кратко этот текст: Хотя у каждого пода есть уникальный IP-адрес, эти IP-адреса не доступны за пределами кластера без использования сервиса. Сервисы позволяют приложениям принимать трафик. Сервисы могут быть по-разному открыты, в зависимости от указанного поля type в ServiceSpec:
ClusterIP (по умолчанию) - открывает доступ к сервису по внутреннему IP-адресу в кластере. Этот тип делает сервис доступным только внутри кластера.
NodePort - открывает сервис на одном и том же порту каждого выбранного узла в кластере с помощью NAT. Делает сервис доступным вне кластера, используя <NodeIP>:<NodePort>. Является надмножеством ClusterIP.
LoadBalancer - создает внешний балансировщик нагрузки в текущем облаке (если это поддерживается) и назначает фиксированный внешний IP-адрес для сервиса. Является надмножеством NodePort.
ExternalName - открывает доступ к сервису с указанным именем (определённое в поле externalName в спецификации) и возвращает запись CNAME. Прокси не используется. Для этого типа требуется версия kube-dns 1.7 или выше."""

output = evaluate(prompt)
print(output)

In [ ]:
#Пример ответа на русском с учетом контекста на английском языке
prompt = """Вопрос: Объясни мне что такое сервис? Ответь на этот вопрос, используя данный текст: 
A Kubernetes Service is an abstraction which defines a logical set of Pods running somewhere in your cluster, that all provide the same functionality. When created, each Service is assigned a unique IP address (also called clusterIP). This address is tied to the lifespan of the Service, and will not change while the Service is alive. Pods can be configured to talk to the Service, and know that communication to the Service will be automatically load-balanced out to some pod that is a member of the Service."""

output = evaluate(prompt)
print(output)